In [9]:
# If not already installed, do: pip install pandas fastparquet
import pandas as pd

URL_DATA = "https://storage.data.gov.my/dashboards/ktmb_timeseries.parquet"

df_dosm = pd.read_parquet(URL_DATA)
if 'date' in df_dosm.columns: df_dosm['date'] = pd.to_datetime(df_dosm['date'])

print(df_dosm)

       service frequency         origin destination       date  passengers
0          ets     daily   All Stations  Alor Setar 2024-03-23         401
1          ets     daily   All Stations  Alor Setar 2024-03-24         233
2          ets     daily   All Stations  Alor Setar 2024-03-25         268
3          ets     daily   All Stations  Alor Setar 2024-03-26         275
4          ets     daily   All Stations  Alor Setar 2024-03-27         403
...        ...       ...            ...         ...        ...         ...
546559  tebrau   monthly  Woodlands CIQ  JB Sentral 2024-01-01      103808
546560  tebrau   monthly  Woodlands CIQ  JB Sentral 2024-02-01       98192
546561  tebrau   monthly  Woodlands CIQ  JB Sentral 2024-03-01      109980
546562  tebrau   monthly  Woodlands CIQ  JB Sentral 2024-04-01      100094
546563  tebrau   monthly  Woodlands CIQ  JB Sentral 2024-05-01       57622

[546564 rows x 6 columns]


## Ridership figures of each month from individual origin and destination

Remove All Stations as origin and destination
Sort ridership figures by highest

In [10]:
df_dosm_only = df_dosm[df_dosm['origin'] != 'All Stations']
df_dosm_only = df_dosm_only[df_dosm_only['destination'] != 'All Stations']
df_dosm_only = df_dosm_only[df_dosm_only['frequency'] == 'monthly']

#df_grouped = df_dosm_only.groupby(['origin', 'destination'])['passengers'].mean().reset_index()

df_dosm_only.sort_values(by='passengers', ascending=False).head(10)


,service,frequency,origin,destination,date,passengers
546541,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-12-01,170436
546542,tebrau,monthly,JB Sentral,Woodlands CIQ,2024-01-01,166177
546536,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-07-01,165269
546544,tebrau,monthly,JB Sentral,Woodlands CIQ,2024-03-01,164339
546534,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-05-01,164231
546532,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-03-01,164023
546537,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-08-01,163419
546535,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-06-01,162689
546539,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-10-01,159068
546538,tebrau,monthly,JB Sentral,Woodlands CIQ,2023-09-01,158958


## Monthly average ridership by origin/destination pair

In [11]:
df_dosm_only = df_dosm[df_dosm['origin'] != 'All Stations']
df_dosm_only = df_dosm_only[df_dosm_only['destination'] != 'All Stations']
df_dosm_only = df_dosm_only[df_dosm_only['frequency'] == 'monthly']


df_grouped = df_dosm_only.groupby(['origin', 'destination'])['passengers'].mean().reset_index()

df_grouped['passengers']= df_grouped['passengers'].astype(int)
df_grouped.sort_values(by='passengers', ascending=False).head(20)


,origin,destination,passengers
1704,JB Sentral,Woodlands CIQ,155383
6611,Woodlands CIQ,JB Sentral,100189
1910,KL Sentral,Butterworth,15360
1338,Butterworth,KL Sentral,14317
1917,KL Sentral,Ipoh,13762
1651,Ipoh,KL Sentral,13655
1903,KL Sentral,Batu Caves,11647
693,Batu Caves,KL Sentral,10425
1194,Bukit Mertajam,KL Sentral,9711
1909,KL Sentral,Bukit Mertajam,9540


## Monthly average ridership by combining return journeys of origin/destination pair
ie. Origin: KLCC - Destination: KL Sentral will be combined with Origin: KL Sentral - Destination: KLCC

In [12]:
import numpy as np

# Sort 'origin' and 'destination' columns
df_grouped[['origin', 'destination']] = pd.DataFrame(np.sort(df_grouped[['origin', 'destination']], axis=1))

# Group by the sorted 'origin-destination-pair' and sum the 'passengers' column
result_df = df_grouped.groupby(['origin', 'destination'])['passengers'].sum().reset_index()

# Combine 'origin' and 'destination' into 'origin-destination-pair'
result_df['origin-destination-pair'] = result_df['origin'] + '-' + result_df['destination']

# Drop the separate 'origin' and 'destination' columns
result_df = result_df[['origin-destination-pair', 'passengers']]

result_df.sort_values(by='passengers', ascending=False).to_csv('ktm_riderships')
# Print the resulting DataFrame
result_df.sort_values(by='passengers', ascending=False).head(10)

,origin-destination-pair,passengers
1596,JB Sentral-Woodlands CIQ,255572
1241,Butterworth-KL Sentral,29677
1548,Ipoh-KL Sentral,27417
663,Batu Caves-KL Sentral,22072
1124,Bukit Mertajam-KL Sentral,19251
1251,Butterworth-Padang Besar,14520
151,Alor Setar-Butterworth,14170
1756,KL Sentral-Klang,12439
282,Arau-Butterworth,12030
714,Batu Gajah-KL Sentral,11586


In [13]:
import pandas as pd
import plotly.express as px

sorted_df  = result_df.sort_values(by='passengers', ascending=True)
# Create an interactive scatterplot

# Create an interactive scatterplot
fig = px.scatter(sorted_df, x='origin-destination-pair', y='passengers', text='passengers',
                 title='Passenger Counts by Origin-Destination Pairs',
                 labels={'origin-destination-pair': 'Origin-Destination Pair'},
                 hover_name='origin-destination-pair')

fig.update_traces(textposition='top center')

# Remove the x-axis labels
fig.update_xaxes(showticklabels=False)

# Customize the appearance of the plot (optional)
fig.update_layout(xaxis_title='Origin-Destination Pair', yaxis_title='Passengers')

# Show the interactive plot
fig.show()

## Group Riderships by only Origin

In [14]:
df_dosm_origin = df_dosm[df_dosm['origin'] != 'All Stations']
df_dosm_origin = df_dosm_origin[df_dosm_origin['destination'] == 'All Stations']
df_dosm_origin = df_dosm_origin[df_dosm_origin['frequency'] == 'monthly']
#df_dosm_origin = df_dosm_origin[df_dosm_origin['date'] == '2023-09-01']

df_dosm_origin = df_dosm_origin.groupby(['origin'])['passengers'].mean().reset_index()
df_dosm_origin['passengers']= df_dosm_origin['passengers'].astype(int)

df_dosm_origin.sort_values(by='passengers', ascending=False).head(13)


,origin,passengers
45,KL Sentral,68066
30,Butterworth,31504
131,Sungai Petani,26542
126,Subang Jaya,25816
2,Alor Setar,24161
5,Arau,23590
14,Batu Caves,23401
26,Bukit Mertajam,21463
38,Ipoh,17840
28,Bukit Tengah,16537


In [15]:
df_dosm_origin.sort_values(by='passengers', ascending=True).head(20)


,origin,passengers
65,Kodiang,9
123,Sri Bintang,14
6,Aur Gading,21
133,Sungai Sirian,22
57,Kg Berkam,23
124,Sri Jaya,27
32,Chicha Tinggi,31
129,Sungai Keladi,32
24,Bukit Betong,38
134,Sungai Tasin,44


In [16]:
df_dosm_origin.to_csv('ktm_origin_ridership_figures.csv')